In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 96.7 MB/s 
     |████████████████████████████████| 895 kB 78.5 MB/s 
     |████████████████████████████████| 3.3 MB 61.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 25 06:55:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Importer les librairies

In [3]:
import glob
import re
import torch

from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch import nn

In [4]:
from google.colab import drive # Se connecter a google drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Telechargement du modèle pré-entrainé gpt2-fr-cased-base

In [104]:
tokenizer = AutoTokenizer.from_pretrained("dbddv01/gpt2-french-small")
model = AutoModelForCausalLM.from_pretrained("dbddv01/gpt2-french-small")

#tokenizer = AutoTokenizer.from_pretrained("gpt2")
#model = AutoModelForCausalLM.from_pretrained("gpt2")


#tokenizer = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-base")
#model = AutoModelForCausalLM.from_pretrained("asi/gpt-fr-cased-base")

loading configuration file https://huggingface.co/dbddv01/gpt2-french-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc6e0d0f90ede5bd59508165ea8c22310ae5a2f8ec0a19c79afb6077e8cf5c92.61cbc5fb2c7aa0655f76d992e92f43eb716e09061c039a6198b2258057850ad6
Model config GPT2Config {
  "_name_or_path": "dbddv01/gpt2-french-small",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary

In [ ]:
model.eval()

# Importer les données

In [126]:
train = sorted(glob.glob('/content/drive/MyDrive/projet-TIA/dataset/train_grimm/*'))
test = sorted(glob.glob('/content/drive/MyDrive/projet-TIA/dataset/test_grimm/*'))

In [29]:
with open("/content/drive/My Drive/train_grimms.txt", 'w') as outfile:
    for fname in train:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [30]:
with open("/content/drive/My Drive/test_grimms.txt", 'w') as outfile:
    for fname in test:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [31]:
# Importer le dataset 
test_data = open("/content/drive/MyDrive/test_grimms.txt", 'rt', encoding='utf-8').read()
train_data = open("/content/drive/MyDrive/train_grimms.txt", 'rt', encoding='utf-8').read()

In [128]:
def clean_text(text):
  pattern = '(page|PAGE|Page)(\s+\|\s+)(ŕ)([0-9]+)(.*)$'
  cleaned = re.sub('\s$','',text, flags=re.MULTILINE)
  p = re.compile(pattern, re.MULTILINE)
  cleaned = p.sub(" ", cleaned)
  return cleaned

In [129]:
test = clean_text(test_data)

In [131]:
train = clean_text(train_data)

In [133]:
train

'Trois femmes avaient été métamorphosées en fleurs et brillaient ainsi dans la campagne. Cependant le charme permettait que l\'une d\'elles retournât chaque nuit dans sa demeure. Il y avait quelque temps qu\'elle subissait cette métamorphose, lorsqu\'elle dit à son mari: "L\'aurore va paraître, et je devrai te quitter de nouveau pour rejoindre mes compagnes et redevenir, comme elles, fleur des champs; mais si tu arrives aujourd\'hui avant midi, et que tu me cueilles, l\'enchantement cessera, et je ne le quitterai plus désormais." Vous me demanderez maintenant comment son mari aura pu la reconnaître, puisque toutes les fleurs étaient pareilles?\nJe vous répondrai: Son mari la reconnut, parce qu\'elle avait passé la nuit à la maison et non dans les champs, et qu\'ainsi la rosée, qui était tombée sur les autres, ne se trouva pas sur elle.\nIl y avait une fois un jeune gaillard qui s\'était engagé dans l\'armée et qui s\'y comporta vaillamment; il était toujours le premier à l\'assaut quan

In [32]:
train_path = "/content/drive/MyDrive/train_grimms.txt"
test_path = "/content/drive/MyDrive/train_grimms.txt"

In [33]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

In [34]:
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file /content/drive/MyDrive/cached_lm_GPT2TokenizerFast_128_train_grimms.txt [took 0.461 s]
Loading features from cached file /content/drive/MyDrive/cached_lm_GPT2TokenizerFast_128_train_grimms.txt [took 0.008 s]


# Initialiser le modèle

In [113]:
#model = AutoModelForCausalLM.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="drive/My Drive/projet-TIA/test", #Le répertoire de sortie
    overwrite_output_dir=True, #écraser le contenu du répertoire de sortie
    num_train_epochs=50, # nombre d'époques d'entraînement
    per_device_train_batch_size=1, # batch size pour l'entrainement
    per_device_eval_batch_size=1,  # batch size pour l'evaluation
    eval_steps = 400, # Nombre d'étapes de mise à jour entre deux évaluations.
    save_steps=800, # après 800 étapes, le modèle est enregistré 
    warmup_steps=10,# nombre d'étapes d'échauffement pour le programmateur de taux d'apprentissage
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [114]:
import gc
gc.collect()

431

In [115]:
import torch
torch.cuda.empty_cache()

# Entrainer le modèle

In [116]:
trainer.train("/content/drive/MyDrive/projet-TIA/test/checkpoint-27200")

Loading model from /content/drive/MyDrive/projet-TIA/test/checkpoint-27200).
***** Running training *****
  Num examples = 1365
  Num Epochs = 50
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 68250
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 19
  Continuing training from global step 27200
  Will skip the first 19 epochs then the first 1265 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1265 [00:00<?, ?it/s]

Step,Training Loss
27500,1.592200
28000,1.612400
28500,1.664800
29000,1.525800
29500,1.509000
30000,1.567200
30500,1.364300
31000,1.394300
31500,1.386500
32000,1.240200


Saving model checkpoint to drive/My Drive/projet-TIA/test/checkpoint-28000
Configuration saved in drive/My Drive/projet-TIA/test/checkpoint-28000/config.json
Model weights saved in drive/My Drive/projet-TIA/test/checkpoint-28000/pytorch_model.bin
Saving model checkpoint to drive/My Drive/projet-TIA/test/checkpoint-28800
Configuration saved in drive/My Drive/projet-TIA/test/checkpoint-28800/config.json
Model weights saved in drive/My Drive/projet-TIA/test/checkpoint-28800/pytorch_model.bin
Saving model checkpoint to drive/My Drive/projet-TIA/test/checkpoint-29600
Configuration saved in drive/My Drive/projet-TIA/test/checkpoint-29600/config.json
Model weights saved in drive/My Drive/projet-TIA/test/checkpoint-29600/pytorch_model.bin
Saving model checkpoint to drive/My Drive/projet-TIA/test/checkpoint-30400
Configuration saved in drive/My Drive/projet-TIA/test/checkpoint-30400/config.json
Model weights saved in drive/My Drive/projet-TIA/test/checkpoint-30400/pytorch_model.bin
Saving model

TrainOutput(global_step=68250, training_loss=0.3902995355068109, metrics={'train_runtime': 2746.7671, 'train_samples_per_second': 24.847, 'train_steps_per_second': 24.847, 'total_flos': 4458295296000000.0, 'train_loss': 0.3902995355068109, 'epoch': 50.0})

In [117]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1365
  Batch size = 1


Perplexity: 1.07


# Sauvegarder le modèle

Une fois la formation terminée, j'enregistrer le modèle en appelant save_model(). Cela enregistrera le modèle formé dans output_dir depuis le TrainingArguments.

In [118]:
trainer.save_model()

Saving model checkpoint to drive/My Drive/projet-TIA/test
Configuration saved in drive/My Drive/projet-TIA/test/config.json
Model weights saved in drive/My Drive/projet-TIA/test/pytorch_model.bin


In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("/content/drive/MyDrive/projet-TIA/test")

# Tester le modèle

Pour tester le modèle, j'utilise pipeline de la bibliotheque Transformers.

In [81]:
# Start every description with a special BOS token
# Start every description with a special BOS token
input_sentence = "Il était une fois un crabe mignon que ses parents avaient appelé le petit Pince. Le petit Pince était d’une beauté à couper le souffle. Il avait six petites pattes roses bonbon parfaitement articulées comme tous les crabes de cocotier bien nés mais sa carapace était spéciale."
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')
# Generate 3 movie descriptions
sample_outputs = model.generate(input_ids, 
                 # Use sampling instead of greedy decoding 
                 do_sample=True, 
                 # Keep only top 50 token with 
                 # the highest probability
                 top_k=50, 
                 # Maximum sequence length
                 max_length=300, 
                 # Keep only the most probable tokens 
                 # with cumulative probability of 95%
                 top_p=0.95, 
                 # Changes randomness of generated sequences
                 temperature=1.9,
                 # Number of sequences to generate                 
                 num_return_sequences=1)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: ignored

In [120]:
story = pipeline('text-generation',
                 model='drive/My Drive/projet-TIA/test', 
                 tokenizer="dbddv01/gpt2-french-small", 
                 max_length=100,
                 top_p=0.95
                 )
set_seed(42)

loading configuration file drive/My Drive/projet-TIA/test/config.json
Model config GPT2Config {
  "_name_or_path": "drive/My Drive/projet-TIA/test",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_ver

In [121]:
text = story("Il était une fois un crabe mignon que ses parents avaient appelé le petit Pince. Le petit Pince était d’une beauté à couper le souffle. Il avait six petites pattes roses bonbon parfaitement articulées comme tous les crabes de cocotier bien nés mais sa carapace était spéciale.")[0]['generated_text']
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Il était une fois un crabe mignon que ses parents avaient appelé le petit Pince. Le petit Pince était d’une beauté à couper le souffle. Il avait six petites pattes roses bonbon parfaitement articulées comme tous les crabes de cocotier bien nés mais sa carapace était spéciale.
Le petit bout de pouvoir s'en empêcher, il prit son aiguille et se mit en route. Au bout de quelque temps, il rencontra un premier château sur lequel il se félicite de l'envie


In [125]:
story("Il était une fois", 
    max_length=150, 
    #do_sample=True,   
    top_k=50, 
    top_p=1,
    num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Il était une fois une femme mais une femme aussi bonne que lui. Quand on lui apprit d\'où elle allaitte, elle n\'avait plus que le mal d\'avoir beaucoup découvert dans la forêt. En revanche, elle savait bien que c\'était elle qui laisserait la vie en repos. Alors, elle pensa qu\'elle ne serait pas sa femme pour lui et s\'écria: "Je ne veux pas être un mari, je voudrais bien une femme. C\'est de donner à mon enfant un pain, et, sans cela, je n\'ai plus besoin de travailler avec une femme." Quand l\'enfant fut né, la femme ne lui pensait pas, mais lui dit: "Je voudrais une femme." Elle'},
 {'generated_text': "Il était une fois très belle honnête et si gracieuse qu'on ne peut plus lui rendre visite! Il arriva que la femme du hérisson se leva en l' voyant de jour et lui dit: « Monsieur, mon enfant! Tu seras peut-être aussi devenir dur et riche maintenant que tu vivras!\n - J'y réfléchirai, répondit-il, et je t'y réfléchirais. Cependant, Mon Dieu, pourquoi pleures-tu? J

In [77]:
# Generate a sample of text

input_sentence = "Il était une fois un crabe mignon que ses parents avaient appelé le petit Pince. Le petit Pince était d’une beauté à couper le souffle. Il avait six petites pattes roses bonbon parfaitement articulées comme tous les crabes de cocotier bien nés mais sa carapace était spéciale."
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

In [78]:
beam_outputs = model.generate(
    input_ids, 
    max_length=100, 
    do_sample=True,   
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=1
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: ignored

In [ ]:
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_outputs[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Il était une fois un crabe mignon que ses parents avaient appelé le petit Pince. Le petit Pince était d’une beauté à couper le souffle. Il avait six petites pattes roses bonbon parfaitement articulées comme tous les crabes de cocotier bien nés mais sa carapace était spéciale. La seule différence était qu’elle avait un cou très court, sans défaut, sans marques mais sans que cela ne le dérange. Elle avait des yeux très bleus, des pattes interminables comme celles des cra
